In [1]:
from tkinter import *
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.metrics import BigramAssocMeasures
from nltk.collocations import BigramCollocationFinder
import tkinter as tk
from PIL import ImageTk,Image
stop_words = stopwords.words('english')
punctuations = list(string.punctuation)
measures = ["teaspoon","tablespoon","cup","ounce","pint","quart","gallon","pound","kilogram","millilitre","litre","inch","centimetre","pinch","gram"]
equivalents = {
't': 'teaspoon',
'tsp':'teaspoon',
'tbsp':'tablespoon',
'c':'cup',
'oz':'ounce',
'pt': 'pint',
'qt': 'quart',
'gal':'gallon',
'lb':'pound',
'kg':'kilograms',
'ml':'millilitre',
'cm':'centimetre',
'gm':'gram'  
}
lemmatizer = nltk.stem.WordNetLemmatizer()
key_word=""
tools=["knife","scissors","oven","pan","mixer","grinder","stirrer","tawa","grater","tongs","masher","fork","strainer","blender","peeler","tongs","rod","spatula"]
instruments=set()
 
# create root window
root = Tk()
 
# root window title and dimension
root.title("Welcome to recipe chatbot")
# Set geometry(widthxheight)
root.geometry('350x200')
 
# adding a label to the root window
lbl = Label(root, text = "Enter a recipie")
lbl.grid()


# adding Entry Field
recipies_name = Entry(root, width=100)
recipies_name.grid(column =15, row =0)

recipies_question = Entry(root, width=100)
recipies_question.grid(column =15, row =5)
 
instrument=[]

def clicked():
    global check
    global kabali
    global overall_ingredient
    global c
    global foo
    recipie=recipies_name.get()
    recipie=recipie.lower()
    recipie=recipie.split(" ")
    ours=""
    for i in recipie:
        ours+=i+"-"
    ours=ours.rstrip(str(-1))
    print(ours)
    url="https://www.vegrecipesofindia.com/"+str(ours.lower())
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    print(url)
    b=soup.find_all("div",class_="wprm-recipe-ingredient-group") 
    outputtext.delete(1.0,tk.END)
    print(len(b))
    if len(b)>0:
        for i in range(0,len(b)):
            print(b[i].text)       
        c=[]
        e=[]
        for i in range(0,len(b)):
            c.append(b[i].text.split("▢"))
        for i in range(0,len(c)):
            del c[i][0]
        ing=""
        overall_ingredient={}
        for i in range (0, len(c)):
            for j in range(0,len(c[i])):
                outputtext.insert(tk.END,c[i][j])
                overall_ingredient.update({parsename(c[i][j]):parse_ingredient(c[i][j])})
        print(overall_ingredient)
        d=soup.find_all("ul",class_="wprm-recipe-instructions")
        kabali=[]
        for i in range(0,len(d)):
            print(d[i].text)
            kabali.append(d[i].text)
        foo=""
        for i in kabali:
            foo+=i
        foo
        kabali=foo.split('.')
        check=[]
        for i in range(0,len(kabali)):
            words = word_tokenize(kabali[i])
            common_words=["step","before","after"]
            words = [word.lower() for word in words]
            words = [word for word in words if word not in stop_words]
            words = [word for word in words if word not in punctuations]
            words = [lemmatizer.lemmatize(word) for word in words]
            words=  [word for word in words if word not in common_words]
            bigram_collocation = BigramCollocationFinder.from_words(words)
            check.append(bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 50))
    else:
        outputtext.insert(tk.END,"Recipie not found. Try other vegetarian recipies like say Mushroom soup")
    return check
def question():
    global check
    global step1
    global key_word
    text=recipies_question.get()
    parsename(text)
    key_tags=["quantity","amount","many","number","much"]
    type_tags=["kind","type"]
    step_words=["step","before","previous","after","next","prepare","current","prep"]
    instru_words=["with","without","cooked","done","used","use"]
    words=text.split(" ")
    outputtext.delete(1.0,tk.END)
    name=""
    srk=[]
    q=text.split(" ")
    for word in words:
        if word in key_tags:
            name=parsename(text)
            try:
                print(overall_ingredient[name]["quantity"], str(overall_ingredient[name]["measurement"]))
                srk.append(overall_ingredient[name]["quantity"])
                srk.append(overall_ingredient[name]["measurement"])
                if name in overall_ingredient.keys():
                    for j in range (0,len(srk)):
                        outputtext.insert(tk.END,srk[j]) 
            except:
                outputtext.insert(tk.END,"Ingredient Not found or error in name of it. Please check name again")
        elif word in type_tags:
            name=parsename(text)
            try:
                srk.append(overall_ingredient[name]["type"])
                if name in overall_ingredient.keys():
                    for j in range (0,len(srk)):
                        outputtext.insert(tk.END,srk[j]) 
            except:
                outputtext.insert(tk.END,"Type not found")

        elif word in step_words:
            if word==step_words[1] or word==step_words[2]:
                key_word="previous"
            elif word==step_words[3] or word==step_words[4]:
                key_word="next"
            else:
                key_word="prepare"
            words1 = word_tokenize(text)
            words1 = [word.lower() for word in words1]
            words1 = [word for word in words1 if word not in stop_words]
            words1 = [word for word in words1 if word not in punctuations]
            words1 = [lemmatizer.lemmatize(word) for word in words1]
            words1=  [word for word in words1 if word not in step_words]
            bigram_collocation = BigramCollocationFinder.from_words(words1)
            qcheck=bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10)
            qcheck
            ctrl=[]
            for j in range(0,len(check)):
                ctrl.append(0)


            for i in range(0,len(check)):
                 for j in range(0,len(check[i])):
                    for k in qcheck:
                        if check[i][j]==k:
                            ctrl[i]+=1
            ctrl
            inter=max(ctrl)
            inter2=ctrl.index(inter)
            if key_word=="previous":
                step=inter2-1
            elif key_word=="next":
                step=inter2+1
            else:
                step=inter2

            print(step)
            print(ctrl)
            print(kabali[step])
            outputtext.delete(1.0,tk.END)
            outputtext.insert(tk.END,kabali[step])
            
        elif word in instru_words:
            if word==instru_words[0]  or instru_words[2] or instru_words[3] or instru_words[4] or instru_words[5] :
                if instrument(text).issubset(instrument(foo)):
                    print("yes, it needs it")
                    outputtext.delete(1.0,tk.END)
                    outputtext.insert(tk.END,"Yes it is required")
                else:
                    print("do not need it")
                    outputtext.delete(1.0,tk.END)
                    outputtext.insert(tk.END,"Nope it is not required")
            if word=="without":
                if instrument(text).issubset(instrument(foo)):
                    print("Nope, we need it")
                    outputtext.delete(1.0,tk.END)
                    outputtext.insert(tk.END,"Nope we need it")                 
                else:
                    print("Yes we can")
                    outputtext.delete(1.0,tk.END)
                    outputtext.insert(tk.END,"Yes we can cook without it")
"""                
        elif word=="serving" or "servings" or "people":
            servings(overall_ingredient,text)
            outputtext.delete(1.0,tk.END)
            outputtext.insert(tk.END,overall_ingredient)
            print(servings(overall_ingredient,text))
            
            
                    
def servings(mama, soph):
    number=re.findall("\d/?.?\d?",soph)
    for name in mama:
        if mama[name]["quantity"].isdigit():
            mama[name]["quantity"]= mama[name]["quantity"]/number
        else:
            continue
    
    return mama
                    
def preprocessing(c):
    words = word_tokenize(c)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stop_words]
    words = [word for word in words if word not in punctuations]
    words = [lemmatizer.lemmatize(word) for word in words]
    return words"""

def instrument(c):
    tools=["knife","scissors","oven","pan","mixer","grinder","stirrer","tawa","grater","tongs","masher","fork","strainer","blender","peeler","tongs","rod","spatula"]
    instruments=set()
    words = word_tokenize(c)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stop_words]
    words = [word for word in words if word not in punctuations]
    measure=[word for word in words if word in measures]
    #print(words)
    instrument=[word for word in words if word in tools]
    if len (instrument)>0:
        instruments.update(instrument)
    #else:
        #continue
    return instruments

                       
def parsename(raw_ingredient):
    
    #quantity=re.findall("\d/?.?\d?",raw_ingredient)
    words = word_tokenize(raw_ingredient)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stop_words]
    words = [word for word in words if word not in punctuations]
    words = [lemmatizer.lemmatize(word) for word in words]
    words = [word for word in words if word.isalpha()]
    words=[word for word in words if word not in measures]
    common_words=["quantity","amount","value","add","optional","frozen","grinding","fresh","chopped","type","kind","whole"]
    words=[word for word in words if word not in common_words]
    #print(words)
    synonyms = []

    for key in equivalents:
        synonyms.append(key)
    for word in words:
        if word in synonyms:
            word = equivalents[word]
    measure=[word for word in words if word in measures]
    pos_tagged_text = nltk.pos_tag(words)
    #print(pos_tagged_text)
    ingredient=[]
    sizes=["small","medium","large","light"]
    ingr=""
    for word in pos_tagged_text:
        if word[1]=="NN":
            ingredient.append(word[0]) 


    if len(ingredient)>1:            
        for i in range(0,len(pos_tagged_text)-1):
            if pos_tagged_text[i][1]=="JJ" and pos_tagged_text[i+1][1]=="NN" :
                ingr=pos_tagged_text[i+1][0]
            elif (pos_tagged_text[i][1]=="NN" and pos_tagged_text[i+1][1]=="NN")and pos_tagged_text[i][1] not in sizes:
                ingr=str(pos_tagged_text[i][0])+"  "+str(pos_tagged_text[i+1][0])
            elif (pos_tagged_text[i][1]=="NN" and pos_tagged_text[i+1][1]=="NN")and pos_tagged_text[i][1] in sizes:
                ingr=str(pos_tagged_text[i+1][0])
            elif pos_tagged_text[i][1]=="VBD" and pos_tagged_text[i+1][1]=="NN":
                ingr=pos_tagged_text[i+1][0]
            elif pos_tagged_text[i][1]=="VB" and pos_tagged_text[i+1][1]=="NN":
                ingr=pos_tagged_text[i+1][0]
    elif len(ingredient)==1:
        ingr=ingredient[0]
    else:
        ingr=None
    
    print(ingr)
    return ingr

def parse_ingredient(raw_ingredient):
    try:
        quantity=re.findall("\d/?.?\d?",raw_ingredient)
    except:
        quantity=None
    words = word_tokenize(raw_ingredient)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stop_words]
    words = [word for word in words if word not in punctuations]
    words = [lemmatizer.lemmatize(word) for word in words]
    words = [word for word in words if word.isalpha()]
    measure=[word for word in words if word in measures]
    words=[word for word in words if word not in measures]
    common_words=["quantity","amount","value"]
    words=[word for word in words if word not in common_words]
    #print(words)
    synonyms = []

    for key in equivalents:
        synonyms.append(key)
    for word in words:
        if word in synonyms:
            word = equivalents[word]
    pos_tagged_text = nltk.pos_tag(words)
    #print(pos_tagged_text)
    ingredient=[]
    for word in pos_tagged_text:
        if word[1]=="NN":
            ingredient.append(word[0])
            ingredient.append(" ")
    ingr=""
    if len(ingredient)>0:
        if len(ingredient)>1:
            for i in ingredient:
                ingr+=str(i)
        else:
            ingr=ingredient[0]
    else:
        ingr=None
    adjective=[]
    for word in pos_tagged_text:
        if word[1]=="JJ":
            adjective.append(word[0])
    if len(quantity)==0:
        quantity="as required"
    req={}
    req["quantity"]=quantity
    req["measurement"]=measure
    req["type"]=adjective
    return req

def ingro():
    outputtext.delete(1.0,tk.END)
    for i in range (0, len(c)):
        for j in range(0,len(c[i])):
            outputtext.insert(tk.END,c[i][j])
            
def process():
    outputtext.delete(1.0,tk.END)
    for i in range(0,len(kabali)):
        outputtext.insert(tk.END,kabali[i])
    
    
            
# button widget with red color text inside
btn = Button(root, text = "click here to get recipie" ,
             fg = "red", command=clicked)
# Set Button Grid
btn.grid(column=5, row=0)
btn1 = Button(root, text = "click here after typing question" ,
             fg = "red", command=question)
# Set Button Grid
btn1.grid(column=5, row=5)

btn2 = Button(root, text = "ingredients click" ,
             fg = "red", command=ingro)
# Set Button Grid
btn2.grid(column=5, row=10)
btn3 = Button(root, text = "process click" ,
             fg = "red", command=process)
# Set Button Grid|
btn3.grid(column=5, row=15)

outputtext = Text(root)
outputtext.grid(column=15, row=100)


# Execute Tkinter
root.mainloop()

mushroom-soup
https://www.vegrecipesofindia.com/mushroom-soup
1
▢ 200 grams white button mushrooms or 2 to 2.5 cups sliced or chopped mushrooms▢ 2 cup small spring onions, sliced or ¼ sliced spring onion whites▢ ½ teaspoon finely chopped garlic or 2 to 3 small to medium garlic cloves - finely chopped▢ ½ inch ginger, chopped or ½ teaspoon finely chopped ginger▢ 1 teaspoon finely chopped celery (optional)▢ ½ teaspoon crushed black pepper or pepper powder or add as required▢ 1 teaspoon gluten free naturally fermented soy sauce or regular soy sauce - add as required▢ 3 cups water▢ 5 tablespoons chopped spring onion greens▢ salt as required
mushroom
spring  onion
clove

celery
pepper
soy  sauce
water
onion  green
salt
{'mushroom': {'quantity': ['200', '2 ', '2.5'], 'measurement': ['gram', 'cup'], 'type': ['white', 'chopped']}, 'spring  onion': {'quantity': ['2 '], 'measurement': ['cup'], 'type': ['small', 'sliced', 'white']}, 'clove': {'quantity': ['2 ', '3 '], 'measurement': ['teaspoon'], 